<h3><B>conda environment:</b> squidpy_env, python v3.9.23

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import squidpy as sq
import scipy.sparse as sp
import scipy.cluster.hierarchy as sch
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.patches as mpatches
import networkx as nx
import seaborn as sns

import os
from pathlib import Path 
import re

from filter_adata import adata_filtered, adata_hightumour, adata_peritumour 
from differential_expression import run_DEG_wilcoxon
from nhood_per_core import make_spatial_neighbors_per_core
from starbars import starbars

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

home_path = '/Users/nabilazulkapeli/Documents/Honours Thesis 2025/nabs_data'
home_path = Path(home_path)
figures_path = home_path / f'results_figures_final'
figures_path = Path(figures_path)
niche_path = figures_path / 'niches'
niche_path.mkdir(parents=True, exist_ok=True)
niche_path = Path(niche_path)


sns.set_theme(context='paper', style='ticks',font='sans-serif')

/opt/anaconda3/envs/squidpy_env/lib/python3.9/site-packages/numba/core/decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


adata and clinical merged, rename the columns if you want
clusters are now annotated with specific and broad cell types


/Users/nabilazulkapeli/Documents/Honours Thesis 2025/final_data/filter_adata.py:194: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adata.obs.loc[melanoma_clusters]
/Users/nabilazulkapeli/Documents/Honours Thesis 2025/final_data/filter_adata.py:213: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adata.obs


low quality cores with <1000 total cells or <100 melanoma cells have been removed
adata_filtered (both high tumour and peritumour cores) has been created
adata_hightumour (only high tumour cores) has been created
adata_peritumour (only peritumour cores) has been created
ready for downstream analysis, you can choose adata_filtered for global analysis and adata_peritumour/adata_hightumour for region-specific analyses


/Users/nabilazulkapeli/Documents/Honours Thesis 2025/final_data/filter_adata.py:290: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_filtered.obs['new_specific_labels'] = adata_filtered.obs['specific_cell_types'].map(new_labels_map)
/Users/nabilazulkapeli/Documents/Honours Thesis 2025/final_data/filter_adata.py:291: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_hightumour.obs['new_specific_labels'] = adata_hightumour.obs['specific_cell_types'].map(new_labels_map)
/Users/nabilazulkapeli/Documents/Honours Thesis 2025/final_data/filter_adata.py:292: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_peritumour.obs['new_specific_labels'] = adata_peritumour.obs['specific_cell_types'].map(new_labels_map)


In [2]:
def plot_spatial_core(adata, core_id, palette=None, size=None):
    coords = adata.obsm["spatial"]
    df = adata.obs.copy()
    df["x"] = coords[:, 0]
    df["y"] = coords[:, 1]

    # select just this core
    core_df = df[df["core_id"] == core_id]

    plt.figure(figsize=(6, 6))

    # 2) overlay niche cells in color
    ax = sns.scatterplot(
        data=core_df,
        x="x", y="y",
        hue="new_specific_labels",
        s=size,
        linewidth=0.1,
        palette=palette
    )

    plt.gca().invert_yaxis()
    plt.axis("off")
    plt.title({core_id})

    # filter legend to show only selected types
    handles, labels = ax.get_legend_handles_labels()
    filtered = [(h, l) for h, l in zip(handles, labels)]
    if filtered:
        handles, labels = zip(*filtered)
        leg = ax.legend(handles, labels, ncols=3, loc='upper center', bbox_to_anchor=(0.5,-.01))
        for handle in leg.legend_handles:
            try:
                handle.set_sizes([10])        # PathCollection
            except AttributeError:
                handle.set_markersize(10)      # Line2D
    plt.savefig(niche_path / f'{core_id}.pdf', bbox_inches='tight')
    plt.show()

In [3]:
global_r_niche_palette = {
    "Melanoma": "#7f7f7f",
    "Endothelial": "#9467bd",
    "Epithelial": "#ff7f0e",
    "cCAF": "#9467bd",
    "iCAF": "#e377c2",
    "Mast": "#2ca02c",
    "Granulocyte": "#7f7f7f",
    "Dendritic": "#9467bd",
    "M1 TAM": "#d62728",
    "M2 TAM": "#9467bd",
    "Ig-TAM": "#1f77b4",
    "Plasmablast": "#1f77b4",
    "Plasma": "#1f77b4",
    "TLS": "#1f77b4",
    "CD4 T": "#17becf",
    "CD8 T": "#d62728"
}

global_nr_niche_palette = {
    "Melanoma": "#7f7f7f",
    "Endothelial": "#9467bd",
    "Epithelial": "#ff7f0e",
    "cCAF": "#9467bd",
    "iCAF": "#e377c2",
    "Mast": "#9467bd",
    "Granulocyte": "#7f7f7f",
    "Dendritic": "#9467bd",
    "M1 TAM": "#1f77b4",
    "M2 TAM": "#9467bd",
    "Ig-TAM": "#1f77b4",
    "Plasmablast": "#9467bd",
    "Plasma": "#1f77b4",
    "TLS": "#1f77b4",
    "CD4 T": "#9467bd",
    "CD8 T": "#1f77b4"
}

In [ ]:
# grab DEG list
global_DEG = run_DEG_wilcoxon(
    adata=adata_filtered,
    response_col='Response',
    celltype_col='new_specific_labels'
)

# neighbourhood enrichment PER CORE
sq_adata_g = make_spatial_neighbors_per_core(adata_filtered, group_key="core_id", delaunay=True)
sq_adata_g.obs['new_specific_labels'] = sq_adata_g.obs['new_specific_labels'].astype('category')
sq.gr.centrality_scores(sq_adata_g, cluster_key='new_specific_labels')
sq.pl.centrality_scores(sq_adata_g, cluster_key="new_specific_labels", figsize=(16, 5))
sq.gr.nhood_enrichment(sq_adata_g, cluster_key="new_specific_labels")

g_sig_DEGs = global_DEG.sort_values("qval").head(40)["gene"].drop_duplicates().tolist()
sq.gr.spatial_autocorr(
    sq_adata_g,
    mode="moran",
    genes=g_sig_DEGs,
    n_perms=100,
    n_jobs=1,
)

# responders vs. non-responders analysis
sq_g_R = sq_adata_g[sq_adata_g.obs.Response=='Responder']
sq_g_NR = sq_adata_g[sq_adata_g.obs.Response=='Non-Responder']

# responders
sq.gr.nhood_enrichment(sq_g_R, cluster_key="new_specific_labels")
sq.pl.nhood_enrichment(
    sq_g_R,
    cluster_key="new_specific_labels",
    figsize=(8, 8),
    title="Global Neighborhood Enrichment in Responders",
)
sq.gr.centrality_scores(sq_g_R, cluster_key='new_specific_labels')

# non-responders
sq.gr.nhood_enrichment(sq_g_NR, cluster_key="new_specific_labels")
sq.pl.nhood_enrichment(
    sq_g_NR,
    cluster_key="new_specific_labels",
    figsize=(8, 8),
    title="Global Neighborhood Enrichment in Non-Responders",
)
sq.gr.centrality_scores(sq_g_NR, cluster_key='new_specific_labels')

# map genes to cell types
gene_to_cell = (
    global_DEG.groupby("gene")["cell_type"]
    .apply(lambda x: list(x.unique()))
    .to_dict()
)

# map integrin complex labels to gene labels
integrin_map = {
    "integrin_aXb2": ["ITGAX"],
    "integrin_aMb2": ["ITGAM"],
    "integrin_aLb2": ["ITGAL"],
}

enrich_df = pd.DataFrame(
    sq_adata_g.uns["new_specific_labels_nhood_enrichment"]["zscore"],
    index=sq_adata_g.obs["new_specific_labels"].cat.categories,
    columns=sq_adata_g.obs["new_specific_labels"].cat.categories,
)

enrich_R  = sq_g_R.uns["new_specific_labels_nhood_enrichment"]
enrich_NR = sq_g_NR.uns["new_specific_labels_nhood_enrichment"]

enrich_R_df = enrich_R["zscore"]
enrich_NR_df = enrich_NR["zscore"]

zscores_log_R = np.sign(enrich_R_df) * np.log1p(np.abs(enrich_R_df))
zscores_log_NR = np.sign(enrich_NR_df) * np.log1p(np.abs(enrich_NR_df))

cell_types = sq_g_R.obs["new_specific_labels"].cat.categories
zscores_df = pd.DataFrame(zscores_log_R, index=cell_types, columns=cell_types)

sns.clustermap(
    zscores_df,
    cmap="RdBu",
    center=0,
    figsize=(10,10),
    linewidths=0.5,  
    annot=False      
)

zscores_df_NR = pd.DataFrame(zscores_log_NR, index=cell_types, columns=cell_types)

sns.clustermap(
    zscores_df_NR,
    cmap="RdBu",
    center=0,
    figsize=(10,10),
    linewidths=0.5,   # add grid lines
    annot=False,       # or True if you want numbers inside cells.
)

In [ ]:
sq.pl.nhood_enrichment(
    sq_g_R,
    cluster_key="new_specific_labels",
    figsize=(8, 8),
    title="Global Neighborhood Enrichment in Responders"
)
plt.savefig(figures_path / 'nhood_R.pdf', dpi=300, bbox_inches='tight')

In [ ]:
sq.pl.nhood_enrichment(
    sq_g_NR,
    cluster_key="new_specific_labels",
    figsize=(8, 8),
    title="Global Neighborhood Enrichment in Non-Responders"
)
plt.savefig(figures_path / 'nhood_NR.pdf', dpi=300, bbox_inches='tight')

In [ ]:
# responder niches
# run clustering on the enrichment matrix
linkage = sch.linkage(zscores_df, method="average")
clusters = sch.fcluster(linkage, t=10, criterion="distance")

# map cluster back to cell types to find niches
niche_assignments_g_R = pd.Series(clusters, index=zscores_df.index)

# non-responder niches
# run clustering on the enrichment matrix
linkage_NR = sch.linkage(zscores_df_NR, method="average")
clusters_NR = sch.fcluster(linkage_NR, t=10, criterion="distance")

# map cluster back to cell types to find niches
niche_assignments_g_NR = pd.Series(clusters_NR, index=zscores_df_NR.index)

# look at niche_assignments_g_R and niche_assignments_g_NR to see niche assignments

niche_assignments_g_NR

<h2><b> Global Responder Niche Plots

In [ ]:
for core in sq_g_R.obs['core_id'].unique():
    plot_spatial_core(sq_g_R, core, size=5, palette=global_r_niche_palette)

<h2><b>Global Non-Responder Niche Plots

In [ ]:
for core in sq_g_NR.obs['core_id'].unique():
    plot_spatial_core(sq_g_NR, core, size=5, palette=global_nr_niche_palette)

In [ ]:
g_niche_NR = ['CD8 T', 'Ig-TAM', 'M1 TAM', 'Plasma', 'TLS']
legend_subset = ['CD8 T', 'Ig-TAM', 'M1 TAM', 'Plasma', 'TLS']


for core in sq_g_NR.obs["core_id"].unique():
    plot_spatial_niche(sq_g_NR, core, g_niche_NR)

In [ ]:
g_niche_stromal = ['Endothelial', 'M2 TAM', 'Mast', 'cCAF', 'Dendritic']
legend_subset = ['Endothelial', 'M2 TAM', 'Mast', 'cCAF', 'Dendritic']

for core in sq_g_R.obs["core_id"].unique():
    plot_spatial_niche(sq_g_R, core, g_niche_stromal)

In [ ]:
g_niche_immune = ['Plasma', 'Plasmablast', 'Ig-TAM']
legend_subset = ['Plasma', 'Plasmablast', 'Ig-TAM']

for core in sq_g_R.obs["core_id"].unique():
    plot_spatial_niche(sq_g_R, core, g_niche_immune)